In [ ]:
import os
os.chdir('/home/sumeet/QDax')

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import functools

from qdax.core.neuroevolution.networks.networks import MLP
from qdax import environments
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.mdp_utils import scoring_function
from qdax.types import Genotype
from qdax.core.containers.mapelites_repertoire import MapElitesRepertoire
from jax._src.flatten_util import ravel_pytree

from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image

In [ ]:
# define the env
env_name='ant_uni'
seed=1111
episode_length=1000
env = environments.create(env_name, episode_length=episode_length)
env_batch_size=1

In [ ]:
# define the MLP architecture
policy_layer_sizes = (128, 128) + (env.action_size,)
policy_network = MLP(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)

In [ ]:
# Init a random key
random_key = jax.random.PRNGKey(seed)
random_key, subkey = jax.random.split(random_key)
keys = jax.random.split(subkey, num=env_batch_size)
fake_batch = jnp.zeros(shape=(env_batch_size, env.observation_size))
init_variables = jax.vmap(policy_network.init)(keys, fake_batch)

# Create the initial environment states
random_key, subkey = jax.random.split(random_key)
keys = jnp.repeat(jnp.expand_dims(subkey, axis=0), repeats=env_batch_size, axis=0)
reset_fn = jax.jit(jax.vmap(env.reset))
init_states = reset_fn(keys)

fake_batch = jnp.zeros(shape=(env_batch_size, env.observation_size))
init_variables = jax.vmap(policy_network.init)(keys, fake_batch)

In [ ]:

def load_archive(random_key):
    cp_path = '/home/sumeet/QDax/experiments/pga_me_ant_uni_testrun_seed_1111/checkpoints/checkpoint_00399/'
    random_key, subkey = jax.random.split(random_key)
    fake_batch = jnp.zeros(shape=(env.observation_size,))
    fake_params = policy_network.init(subkey, fake_batch)

    _, reconstruction_fn = ravel_pytree(fake_params)
    repertoire = MapElitesRepertoire.load(reconstruction_fn=reconstruction_fn, path=cp_path)
    return repertoire

In [ ]:
repertoire = load_archive(random_key=random_key)
best_idx = jnp.argmax(repertoire.fitnesses)
best_fitness = jnp.max(repertoire.fitnesses)
best_bd = repertoire.descriptors[best_idx]
print(
    f"Best fitness in the repertoire: {best_fitness:.2f}\n",
    f"Behavior descriptor of the best individual in the repertoire: {best_bd}\n",
    f"Index in the repertoire of this individual: {best_idx}\n"
)

In [ ]:
random_key, subkey = jax.random.split(random_key)
rand_idx = jax.random.randint(random_key, shape=(1,), minval=0, maxval=len(repertoire.fitnesses))[0]
rand_fitness = repertoire.fitnesses[rand_idx]
rand_bd = repertoire.descriptors[rand_idx]
print(
    f"Random fitness in the repertoire: {rand_fitness:.2f}\n",
    f"Behavior descriptor of the random individual in the repertoire: {rand_bd}\n",
    f"Index in the repertoire of this individual: {rand_idx}\n"
)

In [ ]:
my_params = jax.tree_util.tree_map(
    lambda x: x[rand_idx],
    repertoire.genotypes
)

jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(policy_network.apply)

rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
total_reward = 0
while not state.done:
    rollout.append(state)
    action = jit_inference_fn(my_params, state.obs)
    state = jit_env_step(state, action)
    total_reward += state.reward

print(f"The trajectory of this individual contains {len(rollout)} transitions.")
print(f'{total_reward=}')

HTML(html.render(env.sys, [s.qp for s in rollout[:500]]))